# Is in Colab, this is the config enviroment

In [88]:
if 'google.colab' in str(get_ipython()):
  import os

  if ((not os.path.isdir('./corrida1')) and (not os.path.isdir('./corrida2'))):
    
    !pip install googledrivedownloader
    !pip install wandb

    os.environ["WANDB_API_KEY"] = "cd30e32b876086a7c69da7467b0570a036bb6a0d"

    import shutil  
    from google_drive_downloader import GoogleDriveDownloader as gdd
    
    gdd.download_file_from_google_drive(file_id='1jTG-5XSui9vWlhqE9jzswUDU9vhzburs',
                                      dest_path='./corrida2.zip',
                                      unzip=True)

    gdd.download_file_from_google_drive(file_id='1gBR-TiZIeXu6yM_cI0_iWSdzDpyXNpgw',
                                  dest_path='./corrida1.zip',
                                  unzip=True)

    os.remove('./corrida1.zip')
    os.remove('./corrida2.zip')

else:

  print('Not running on CoLab')

In [89]:
#from google.colab import drive
#drive.mount('/content/drive')

# Imports

In [90]:
# General import of torch.
import torch
# Import for graph blocks of torch.
import torch.nn as nn
# Import the models library, to get the model to be used.
import torchvision.models as models
# Import optim library, to get the optimizer to be used.
import torch.optim as optim
# Import torchvision, to manage the input data.
import torchvision
# To apply transformations to the data (when loaded).
import torchvision.transforms as transforms

import torchvision.transforms.functional as TF

import albumentations as A

# To calculate softmax.
from torch.nn import Softmax

from torch.utils.data import Dataset, DataLoader


# Metrics zero_division
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

# General imports.
import os
import time
import wandb
import random
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from pathlib import Path
from numpy import float32
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

# Enviroment configuration.

## Config variables

In [91]:
softmax = Softmax(dim=1)

## Config for metrics

In [92]:
# Input size of the model.
inputSize = 'inputSize'
# Output size of the model.
outputSize = 'outputSize'
# Batch size.
batchSize = 'batchSize'
# Learning rate.
learningRate = 'learningRate'
# Class names.
classes = 'classes'
# Class IDs.
classesID = 'classesIDs'
# Class IDs.
classesIDs = 'classesIDs'
# Number of classes to classify.
classesLen = 'classesLen'
# Device to be used, prefer cuda, if available.
device = 'device'
# Group type, for wandb.
groupType = 'groupType'
# Config run.
configRun = 'configRun'

config = {
    inputSize    : 224,
    outputSize   : 39,
    batchSize    : 128,
    learningRate : 0.001,
    classes : [
        'Apple - Apple scab',
        'Apple - Black rot',
        'Apple - Cedar apple rust',
        'Apple - Healthy',
        'Background without leaves',
        'Blueberry - Healthy',
        'Cherry - Healthy',
        'Cherry - Powdery mildew',
        'Corn - Cercospora',
        'Corn - Common rust',
        'Corn - Healthy',
        'Corn - Northern Leaf Blight',
        'Grape - Black rot',
        'Grape - Esca',
        'Grape - Healthy',
        'Grape - Leaf blight',
        'Orange - Haunglongbing',
        'Peach - Bacterial spot',
        'Peach - Healthy',
        'Pepper bell - Bacterial spot',
        'Pepper bell - healthy',
        'Potato - Early blight',
        'Potato - Healthy',
        'Potato - Late blight',
        'Raspberry - healthy',
        'Soybean - Healthy',
        'Squash - Powdery mildew',
        'Strawberry - Healthy',
        'Strawberry - Leaf scorch',
        'Tomato - Bacterial spot',
        'Tomato - Early blight',
        'Tomato - Healthy',
        'Tomato - Late blight',
        'Tomato - Leaf Mold',
        'Tomato - Septoria leaf spot',
        'Tomato - Spider mites',
        'Tomato - Target Spot',
        'Tomato - Mosaic virus',
        'Tomato - Yellow Leaf Curl Virus'
    ],
    classesID : [i + 1 for i in range(39)],
    classesLen : 39,
    device : torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
}

# Wandb enviroment variables
os.environ['WANDB_SILENT'] = 'true'

## Executions config

In [93]:
exe_load_check_point = "load_check_point"
exe_freeze_check_point = "freeze_check_point"
exe_check_point_path = "check_point_path"
exe_can_epochs = "can_epochs"
exe_save_check_point_path = "save_check_point_path"
exe_execution_name = "execution_name"
exe_data_path = "data_path"
exe_group_type = "groupType"


# config_freeze = {
#     exe_load_check_point: True,
#     exe_freeze_check_point: True,
#     exe_check_point_path: "./"
#     exe_can_epochs: 1,
#     exe_save_check_point_path: "./",
#     exe_execution_name: "name1",
#     exe_data_path: "./"
# }

# config_unfreeze = {
#     exe_load_check_point: True,
#     exe_freeze_check_point: False,
#     exe_check_point_path: "./"
#     exe_can_epochs: 1,
#     exe_save_check_point_path: "./",
#     exe_execution_name: "name2",
#     exe_data_path: "./"
# }

config_no_load = {
    exe_load_check_point: False,
    exe_freeze_check_point: False,
    exe_check_point_path: "",
    exe_can_epochs: 10,
    exe_save_check_point_path: "/content/corridaTest",
    exe_execution_name: "corridaTest",
    exe_data_path: "/content/corrida1",
    exe_group_type: "Corrida 1"
}

# executions_configs = [config_freeze,
#                       config_unfreeze,
#                       config_no_load]

executions_configs = [config_no_load]

# Models

## Unet

In [94]:
def kernel_size(initial_value, iterations, division_value=2):
  for i in range(iterations):
    initial_value//=division_value
  return initial_value

In [95]:
class DoubleConv(nn.Module):
  def __init__ (self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
        #Here we add Batch Normalization to improve the paper's model
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    )

  def forward(self, x):
    return self.conv(x)

In [96]:
class UNET(nn.Module):
  #features represent the original paper dimensions.
  def __init__(self, 
               in_channels=3, 
               out_channels=1, 
               features=[16, 32, 64, 128, 256, 512], 
               linearFeature=1024,
               isOnlyEncoder=False):
    
    super(UNET, self).__init__()
    
    self.linearFeature = linearFeature
    self.isOnlyEncoder = isOnlyEncoder
    #Encoder part
    self.downs = nn.ModuleList()
    #Decoder part
    self.ups = nn.ModuleList()
    #Pool
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.features = features
    self.kernelHeight = kernel_size(config[inputSize], len(self.features))
    self.kernelWidth = kernel_size(config[inputSize], len(self.features))
    self.kernelSize = self.kernelHeight * self.kernelWidth

   #Create a list of contracting path
    for feature in features:
      self.downs.append(DoubleConv(in_channels, feature))
      in_channels = feature

   #Create a list of expansive path
    for feature in reversed(features):
      self.ups.append(
         #Featur2 * 2 is to create the 1024 dim
         nn.ConvTranspose2d(feature *2, feature, kernel_size=2, stride=2,)
     )
      self.ups.append(DoubleConv(feature * 2, feature))
  
    self.bottleneck = DoubleConv(features[-1], features[-1]*2)

    self.linear1 = nn.Linear(self.features[-1]*2*self.kernelSize,  self.linearFeature)
    self.linear2 = nn.Linear(self.linearFeature, features[-1]*2*self.kernelSize)

    self.final_conv = nn.Conv2d(self.features[0], out_channels, kernel_size=1)

  def forward(self, x):
   
    #For save the connections with the up part
    skip_connections = []
    for down in self.downs:
      x = down(x)
      skip_connections.append(x)
      x = self.pool(x)
    #Here is the bottom part of the net
    x = self.bottleneck(x)
    x = torch.flatten(x, start_dim=1)
    x = self.linear1(x)
    #Check if we are using only de encoder part
    if self.isOnlyEncoder:
      return x
    x = self.linear2(x)
    x = x.reshape((x.shape[0], self.features[-1] *2 , self.kernelHeight, self.kernelWidth))
    #Start the up part
    #Reverse list
    skip_connections = skip_connections[:: -1]

    #Step of two because we use up and doubleconv
    #0 is the up
    #1 is the double conv
    for idx in range (0, len(self.ups), 2):
      x = self.ups[idx](x)
      #Divide idx by 2 for going liner with the skip connections
      skip_connection = skip_connections[idx//2]

      #General solutions for image tha not are divisibles
      if x.shape != skip_connection.shape:
        #Take the H and W, skip the Batch Size and Channels
        x = TF.resize(x, size=skip_connection.shape[2:])
      #Add the skip connection
      concat_skip = torch.cat((skip_connection, x), dim=1)
      # (e.g 0 +1 for the doubleconv)
      x = self.ups[idx+1](concat_skip)
    
    return self.final_conv(x)

## Classification

In [97]:
class ClassifierUnet(nn.Module):
  #features represent the original paper dimensions.
  def __init__(self, 
               out_channels=1, 
               linear=[1024, 250, 150, 100, 50], 
               unetModel=None):
    
    super(ClassifierUnet, self).__init__()

    self.unetModel = unetModel
    self.linear_layers = nn.Sequential()

    #Create a list of linear classificator
    actual_linear = self.unetModel.linearFeature

    for i in range(len(linear)):
      self.linear_layers.add_module(str(i)+"_linear", nn.Linear(actual_linear, linear[i]))
      self.linear_layers.add_module(str(i)+"_activ", nn.ReLU())
      
      actual_linear = linear[i]
    
    self.linear_layers.add_module(str(i+1)+"_linear", nn.Linear(actual_linear, out_channels))


  def forward(self, x):
      x = self.unetModel(x)
      return self.linear_layers(x)

# Utils

## Miscellaneous functions.

In [98]:
# About metrics.
# Metric dictionary keys.
# For preprocessing.
_loss        = 'Loss'
_groundtruth = 'Groundtruth'
_logits      = 'Logits'
# For postprocessing.
_probabilities   = 'Probabilities'
_predictions     = 'Predictions'
_confusionMatrix = 'Confusion matrix'
_accuracyClass   = 'Accuracy class'
_accuracy        = 'Accuracy'
_recall          = 'Recall'
_precision       = 'Precision'
_f1              = 'F1'
_auc             = 'AUC'
# For torch save
_model           = 'Model State Dic'
_optimizer       = 'Optimizer State Dic'
_criterion       = 'Criterion'
_epoch           = 'Epoch'
_runID           = 'RunID'
_metricsTrain    = 'Resulting metrics (training)'
_metricsTest     = 'Resulting metrics (testing)'

_metricsPrint = [_accuracy, _recall, _precision, _f1, _auc]

# Get a clean dictionary for the metrics.
def getMetricsDict():
    return {
        _loss          : torch.tensor(0.),
        _groundtruth   : torch.tensor([]),
        _logits        : torch.tensor([])
    }

# Function used to update the dictionary of resulting metrics.
def updateRunningMetrics(logits, groundtruth, loss, batchAmount, metricsResults):
    # Accumulate the loss.
    metricsResults[_loss] += loss.cpu() / batchAmount
    # Accumulate the groundtruth and the logits.
    metricsResults[_groundtruth] = torch.cat((metricsResults[_groundtruth], groundtruth.cpu())) 
    metricsResults[_logits] = torch.cat((metricsResults[_logits], logits.cpu()))

# Function used to process the dictionary of resulting metrics (make final calculations).
def processRunningMetrics(metricsResults):
    # Detach the other values in the dictionary.
    metricsResults[_loss] = metricsResults[_loss].detach()
    metricsResults[_groundtruth] = metricsResults[_groundtruth].detach()
    metricsResults[_logits] = metricsResults[_logits].detach()
    # Save in the dictionary the probabilities and the predictions.
    metricsResults[_probabilities] = softmax(metricsResults[_logits]).detach()
    metricsResults[_predictions] = torch.argmax(metricsResults[_probabilities], axis=1).detach()

    # Get Groundtruth (as numpy).
    groundtruth = metricsResults[_groundtruth].detach().numpy()
    # Get probabilities (as numpy).
    probs = metricsResults[_probabilities].detach().numpy()
    # Get predictions (as numpy).
    preds = metricsResults[_predictions].detach().numpy()

    # Calculate confusion matrix (normalized).
    confusionMatrix = confusion_matrix(groundtruth, preds)
    metricsResults[_confusionMatrix] = torch.tensor(confusionMatrix)

    # Calculate accuracy by class.
    confusionMatrix = confusionMatrix.astype('float') / confusionMatrix.sum(axis=1)[:, np.newaxis]
    metricsResults[_accuracyClass] = torch.tensor(confusionMatrix.diagonal())

    # Calculate accuracy.
    metricsResults[_accuracy] = torch.tensor(accuracy_score(groundtruth, preds))
    # Calculate recall.
    metricsResults[_recall] = torch.tensor(recall_score(groundtruth, preds, average='macro', zero_division=0))
    # Calculate precision.
    metricsResults[_precision] = torch.tensor(precision_score(groundtruth, preds, average='macro', zero_division=0))
    # Calculate F1.
    metricsResults[_f1] = torch.tensor(f1_score(groundtruth, preds, average='macro', zero_division=0))
    # Calculate AUC.
    metricsResults[_auc] = torch.tensor(roc_auc_score(groundtruth, probs, multi_class='ovr'))

# Pretty print the metrics dictionaries.
def printMetricsDict(metricsResults):
    # All metrics to print
    metricPrints = []

    # Format the loss.
    lossPrint = 'Loss: {:.4f}'.format(metricsResults[_loss])
    metricPrints.append(lossPrint)

    # Format the the remaining metrics.
    baseMetricString = '{}: {:1.4f}'
    for metric in _metricsPrint:
        metricPrints.append(baseMetricString.format(metric, metricsResults[metric]))

    print(', '.join(metricPrints))

# This functions process an metrics result dictionary for wandb. Is necessary to indicte
#   the metrics origin, training or testing.
def processMetricsWandb(metricsResults, training=False):
    # Get the prefix to log on wandb, the keys must be different.
    resultsType = 'training' if training else 'testing'

    # Confusion matrix as a heat map.
    _confusionMatrixHM = 'Confusion matrix (heat map)'

    # All the wandb keys are based in the original metrics results keys.
    lossKey = '{} ({})'.format(_loss, resultsType)
    metricsKeys = ['{} ({})'.format(_metric, resultsType) for _metric in _metricsPrint]
    accuracyClassKeys = ['{} accuracy ({})'.format(_class, resultsType) for _class in config[classes]]
    confusionMatrixKey = '{} ({})'.format(_confusionMatrix, resultsType)
    confusionMatrixHMKey = '{} ({})'.format(_confusionMatrixHM, resultsType)

    # Get the confusion matrix for wandb.
    data = []
    for i in range(config[classesLen]):
        for j in range(config[classesLen]):
            data.append([config[classes][i], config[classesID][i], config[classes][j], config[classesID][j], metricsResults[_confusionMatrix][i, j].item()])

    confusionMatrix = wandb.Table(columns=["Actual", "Actual (id)", "Predicted", "Predicted (id)", "nPredictions"], data=data)

    confusionMatrixHM = wandb.plots.HeatMap(config[classes], config[classes], metricsResults[_confusionMatrix].numpy(), show_text=False)

    # Make the dictionary for wandb and store the values.
    wandbDict = {
        lossKey              : metricsResults[_loss].item(),
        confusionMatrixKey   : confusionMatrix,
        confusionMatrixHMKey : confusionMatrixHM
    }
    for i in range(len(metricsKeys)):
        wandbDict[metricsKeys[i]] = metricsResults[_metricsPrint[i]].item()
    for i in range(config[classesLen]):
        wandbDict[accuracyClassKeys[i]] = metricsResults[_accuracyClass][i].item()

    # Return, to log later.
    return wandbDict

# Get the metrics dictionaries for wandb and log them.
def logMetricsWandb(trainMetricsResults, testMetricsResults):
    # Get both dictionaries for wandb.
    wandbTrainDict = processMetricsWandb(trainMetricsResults, training=True)
    wandbTestDict  = processMetricsWandb(testMetricsResults, training=False)

    # Merge the dictionaries.
    wandbDict = {**wandbTrainDict, **wandbTestDict}

    # Log on wandb
    wandb.log(wandbDict)

# Function used to save the model and the metrics.
def saveEpochData(trainMetricsResults, testMetricsResults, model, optimizer, criterion, epoch, rootPath, runID):
    # Create a dir for the current epoch.
    runDir = os.path.join(os.getcwd(), rootPath, str(epoch))
    Path(runDir).mkdir(parents=True, exist_ok=True)

    # Path
    savePath = os.path.join(runDir, 'model.pth')

    # Make dict for torch.save
    saveDict = {
        _model     : model.state_dict(),
        _optimizer : optimizer.state_dict(),
        _criterion : criterion,
        _epoch     : epoch,
        _runID     : runID
    }

    # Save both metrics, for train and test.
    metricsResults = {
        _metricsTrain : trainMetricsResults,
        _metricsTest  : testMetricsResults
    }

    # Merge the save dict with the metricsResults dict.
    saveDict = {**metricsResults, **saveDict}

    # Save
    torch.save(saveDict, savePath)

    for directory in os.listdir(os.path.join(os.getcwd(), rootPath)):
      if os.path.join(os.getcwd(), rootPath, directory) != runDir:
        shutil.rmtree(os.path.join(os.getcwd(), rootPath, directory))


## Loader function.
Should return the training loader and test loader, a iterable object by batches.

In [99]:
# Transformation definitions.
transformTrain = transforms.Compose([
        transforms.RandomResizedCrop(config[inputSize]),  # This one does a resize (it cuts randomly, it doesn't keep the whole image).
        transforms.RandomHorizontalFlip(),                # Flip the image horizontally randomly.
        transforms.ToTensor(),                            # Make the image a tensor.
        transforms.Normalize(
            [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Expected normalization for pretrained pytorch models.
    ])
transformTest = transforms.Compose([
        transforms.Resize(config[inputSize]),             # Resize the image, keeping all pixels.
        transforms.CenterCrop(config[inputSize]),         # Cut the image in the center.
        transforms.ToTensor(),                            # Make the image a tensor.
        transforms.Normalize(
            [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Expected normalization for pretrained pytorch models.
    ])

# Function used to get the data loaders.
# A folder with two folders inside called train and test is expected as a rootPath.
def getLoaders(rootPath):
    trainPath = os.path.join(rootPath, 'labelTrain')
    testPath  = os.path.join(rootPath, 'labelTest')

    # Get the training and test data, apply the transformations.
    trainset = torchvision.datasets.ImageFolder(root=trainPath, transform=transformTrain)
    testset  = torchvision.datasets.ImageFolder(root=testPath,  transform=transformTest)

    # Get the loaders, to iterate the data through batches.
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=config[batchSize], shuffle=True, num_workers=2)
    testloader  = torch.utils.data.DataLoader(testset,  batch_size=config[batchSize], shuffle=True, num_workers=2)
    
    return trainloader, testloader

## Training method.
This method takes care of a single training pass. Another function call this one multiple times.

In [100]:
def trainEpoch(dataloader, model, criterion, optimizer):
    # Metrics for training.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(True):

        # Indicate that the model is going to be trained.
        model.train()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for training.
        for batch in dataloader:
            # Train the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(config[device]), batch[1].to(config[device])
            # Zero the gradient parameters.
            optimizer.zero_grad()
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Calculates the derivatives of the parameters that have a gradient.
            loss.backward()
            # Update the parameters based on the computer gradient.
            optimizer.step()
            # Metrics for the training set.
            updateRunningMetrics(outputs, labels, loss, loaderLen, metricsResults)

    return metricsResults

## Evaluation method.
This method evaluates the model for a specified dataset.

In [101]:
def evaluate(dataloader, model, criterion):
    # Metrics for testing.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(False):

        # Indicate that the model is going to be evaluated.
        model.eval()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for testing.
        for batch in tqdm(dataloader, leave=False):
            # Test the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(config[device]), batch[1].to(config[device])
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Metrics for the testing set.
            updateRunningMetrics(outputs, labels, loss, loaderLen, metricsResults)

    return metricsResults

## Trainining and evaluate method.
For the specific purpose of this project, in each epoch we evaluate metrics for each data set (training and testing) in each epoch, this method simplifies the process. 

In [102]:
def trainAndEvaluate(trainloader, testloader, model, criterion, optimizer, savePath, epochsInit, epochs, runID):

    startTimeTotal = time.time()

    for epoch in tqdm(range(epochsInit, epochs + 1)):
        
        # Train the model.
        trainMetricsResults = trainEpoch(trainloader, model, criterion, optimizer)
        processRunningMetrics(trainMetricsResults)

        # Evaluate the model.
        testMetricsResults = evaluate(testloader, model, criterion)
        processRunningMetrics(testMetricsResults)

        # Log on wandb
        logMetricsWandb(trainMetricsResults, testMetricsResults)

        # Save model and metrics for the epochs.
        saveEpochData(trainMetricsResults, testMetricsResults, model, optimizer, criterion, epoch, savePath, runID)

        # Print the results.
        if epoch % 1 == 0:
            print('**', '[', 'Epoch ', epoch, ']', '*' * 48, sep='')
            print('\tTraining results:', end=' ')
            printMetricsDict(trainMetricsResults)
            print('\t Testing results:', end=' ')
            printMetricsDict(testMetricsResults)
        
    # Print time
    print('Epochs terminados')
    print("--- %s seconds ---" % (time.time() - startTimeTotal))

## General method for execution.

In [120]:
def loadCheckpoint(model, optimizer, criterion, loadPath):
    # Load the checkpoint.
    checkpoint = torch.load(loadPath)
    # Load model.
    model.load_state_dict(checkpoint[_model])
    # Load optimizer.
    optimizer.load_state_dict(checkpoint[_optimizer])
    # Load criterion.
    criterion = checkpoint[_criterion]
    # Run ID.
    runID = checkpoint[_runID]
    # Set epoch.
    epochsInit = checkpoint[_epoch] + 1

    return model, optimizer, criterion, epochsInit+1, runID

def executeTest(dataPath, runName, net, savePath, epochs):
    if not os.path.exists(savePath):
      loadPath = None
    else:
      if len(os.listdir(savePath)) == 0:
        loadPath = None
      else:
        epochs_check = []

        for i in os.listdir(savePath):
          epochs_check.append(int(i))

        loadPath = os.path.join(savePath, str(min(epochs_check)), "model.pth")

        
    # Get criterion and optimizer.
    # Optimizer and the loss funtion used to train the model.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adagrad(net.parameters(), lr=config[learningRate])

    # Check if required to load a checkpoint.
    resumeF = False
    if loadPath != None:
        net, optimizer, criterion, epochsInit, runID = loadCheckpoint(net, optimizer, criterion, loadPath)
        net.to(config[device])
        resumeF = True
    else:
      epochsInit = 0
      runID = wandb.util.generate_id()

    os.environ["WANDB_RUN_ID"] = runID

    # Get the loaders.
    trainloader, testloader = getLoaders(dataPath)

    # Init wandb
    run = wandb.init(id=runID, project='Classifier-UNET-RESNET', entity='tecai', config=config, name=runName, resume=resumeF)

    # Train and evaluate
    trainAndEvaluate(trainloader, testloader, net, criterion, optimizer, savePath, epochsInit, epochs, runID)

    # Finish wandb
    run.finish()

# Train

In [124]:
for execution_config in executions_configs:
  
  config[groupType] = execution_config[exe_group_type]
  config[configRun] = execution_config
  
  unet = UNET(in_channels=3, out_channels=3, isOnlyEncoder=True).to(config[device])

  if execution_config[exe_load_check_point]:
    load_checkpoint(torch.load(execution_config[exe_check_point_path]), unetTest)

  if execution_config[exe_freeze_check_point]:
    unet.eval()
    
  classifier_UNET = ClassifierUnet(out_channels=config[classesLen], unetModel=unet).to(config[device])
  
  executeTest(execution_config[exe_data_path],
              execution_config[exe_execution_name],
              classifier_UNET,
              execution_config[exe_save_check_point_path],
              execution_config[exe_can_epochs])

**[Epoch 6]************************************************
	Training results: Loss: 2.9008, Accuracy: 0.1687, Recall: 0.0729, Precision: 0.0285, F1: 0.0324, AUC: 0.7622
	 Testing results: Loss: 3.0905, Accuracy: 0.1777, Recall: 0.0724, Precision: 0.0230, F1: 0.0300, AUC: 0.7430


KeyboardInterrupt: ignored